In [1]:
import pandas as pd
import os
import warnings

import preprocessing_tools as pr
from cleanedNames import KeyProc

cn = KeyProc()
warnings.filterwarnings('ignore')

%reload_ext autoreload

Порядок работы: 

     1) Обрабатывается текст(препроцессинг, лемматизация) 
     2) Предсказывается рубрика. Сейчас  три рубрики: "Интернет и сми", "Бизнес", "Спорт" 
     3) В зависимости от рубрики, подключается нужная модель BigArtm и предсказывает,  подходящий по смыслу, топик 

In [2]:
dict_rub = {'Интернет и СМИ':'Internet_i_SMI','Бизнес':'Biznes','Спорт':'Sport'}

In [3]:
# Текст  новости с сайта Лента из рубрики "Интернет и сми" 19:39, 26 февраля 2019
text = '''Японский геймдизайнер Хидэо Кодзима опубликовал фотографию водки,чем привлек 
внимание русскоязычных комментаторов. Пост с алкоголем глава студии по разработке компьютерных игр 
выложил в Instagram.Россияне оставили множество комплиментов на странице Кодзимы. Большинство комментариев
содержали слова «гений» и «гениальный», отсылая к мему про геймдизайнера.Юзеры также начали использовать эпитет
«царь» по отношению к основателю игровой студии из-за марки водки на снимке.Один из пользователей с ником rodion_obrien
предложил выдвинуть Кодзиму в президенты. Некоторые комментаторы предположили, что фотографии — это намек на появление
в будущих играх русской озвучки.Накануне, 25 февраля, Кодзима опубликовал в Twitter несколько снимков с подарками от
российского иллюстратора Ильи Кувшинова. Тот посетил студию Kojima Productions и одарил ее сотрудников картинами своего
авторства.В январе 2018 года Кодзима создал аккаунт в 
Instagram, после этого русскоязычные юзеры оставили под публикациями похвалу в адрес геймдизайнера.'''

In [4]:
# Обработка текста  для предсказания рубрики
text = pr.clean_text(text)
text = cn.replaceKeywords(text)
text = pr.lemmatization(text)
print(text)


['японский', 'геймдизайнер', 'хидэо', 'кодзима', 'опубликовать', 'фотография', 'водкич', 'привлечь', 'внимание', 'русскоязычный', 'комментатор', 'пост', 'алкоголь', 'глава', 'студия', 'разработка', 'компьютерный', 'игра', 'выложить', 'instagram', 'россиянин', 'оставить', 'множество', 'комплимент', 'страница', 'кодзима', 'большинство', 'комментарий', 'содержимый', 'гений', 'гениальный', 'отсылать', 'мема', 'геймдизайнераюзер', 'начать', 'использовать', 'эпитет', 'царь', 'основатель', 'игровой', 'студия', 'изз', 'марка', 'водка', 'снимкеодин', 'пользователь', 'ник', 'rodionobrien', 'предложить', 'выдвинуть', 'кодзима', 'некоторый', 'комментатор', 'предположить', 'фотография', 'намёк', 'появление', 'будущий', 'игра', 'русский', 'озвучкинакануне', 'кодзима', 'опубликовать', 'twitter', 'снимок', 'подарок', 'российский', 'иллюстратор', 'илья', 'кувшинов', 'посетить', 'студия', 'kojima', 'productions', 'одарить', 'сотрудник', 'картина', 'авторствать', 'кодзима', 'создать', 'аккаунт', 'instagr

In [6]:
# Загружаем модели для предсказания рубрики

PATHMODEL = 'data/model/' 
PATH_BIGARTM_MODEL = 'data/model/bigartm/'

from sklearn.externals import joblib

vect = joblib.load(PATHMODEL+'vect_file.pkl')
linersvc = joblib.load(PATHMODEL+'model_for_18.pkl')
print('load ok')

load ok


In [7]:
text_pred = [','.join(text)]
print (text_pred)
text_vect_train = vect.transform(text_pred)
pred = linersvc.predict(text_vect_train)


['японский,геймдизайнер,хидэо,кодзима,опубликовать,фотография,водкич,привлечь,внимание,русскоязычный,комментатор,пост,алкоголь,глава,студия,разработка,компьютерный,игра,выложить,instagram,россиянин,оставить,множество,комплимент,страница,кодзима,большинство,комментарий,содержимый,гений,гениальный,отсылать,мема,геймдизайнераюзер,начать,использовать,эпитет,царь,основатель,игровой,студия,изз,марка,водка,снимкеодин,пользователь,ник,rodionobrien,предложить,выдвинуть,кодзима,некоторый,комментатор,предположить,фотография,намёк,появление,будущий,игра,русский,озвучкинакануне,кодзима,опубликовать,twitter,снимок,подарок,российский,иллюстратор,илья,кувшинов,посетить,студия,kojima,productions,одарить,сотрудник,картина,авторствать,кодзима,создать,аккаунт,instagram,русскоязычный,юзер,оставить,публикация,похвала,адрес,геймдизайнер']


In [8]:
pred[0]

'Интернет и СМИ'

In [9]:
PATH = 'data/all/'
file = dict_rub[pred[0]]

In [10]:
%%time
df = pd.read_csv(PATH+file+'.csv.bz2',usecols=['date','text'])
df = df[df.text!='text']
df['year'] = df['date'].apply(lambda x: int(x.split(' ')[0].split('-')[0]))
df['month'] = df['date'].apply(lambda x: int(x.split(' ')[0].split('-')[1]))
df = df[df.year>2007]
df['docID'] = list(range(df.shape[0]))
df = df[['docID','year','month','text']]
print(df.head())

      docID  year  month                                               text
8928      0  2008      1  C 1 января 2008 года на территории Азербайджан...
8929      1  2008      1  Специалисты по связям с общественностью америк...
8930      2  2008      1  Состояние знаменитых наскальных росписей пещер...
8931      3  2008      1  86 журналистов погибли на планете в 2007 году....
8932      4  2008      1  Европейская система спутниковой навигации заго...
CPU times: user 23.9 s, sys: 180 ms, total: 24.1 s
Wall time: 24.1 s


In [11]:
df.index = df['docID']
del df['docID']
df.head()

,year,month,text
docID,,,
0,2008,1,C 1 января 2008 года на территории Азербайджан...
1,2008,1,Специалисты по связям с общественностью америк...
2,2008,1,Состояние знаменитых наскальных росписей пещер...
3,2008,1,86 журналистов погибли на планете в 2007 году....
4,2008,1,Европейская система спутниковой навигации заго...


In [13]:
# Загружаем  модель для определения топика
import sys
sys.path.append('/home/midzay/Mlerning/bigartm/python')
import artm



In [14]:
%%time
model = artm.load_artm_model(f'/home/midzay/Mlerning/GitHUb/topic_news/bigartm/data/model/bigartm/{dict_rub[pred[0]]}/')

CPU times: user 367 ms, sys: 35.7 ms, total: 402 ms
Wall time: 397 ms


In [15]:
%%time
vwpath = f'data/vwpath/999_input_bigartm.vw'

with open(vwpath, 'w') as fp:
    fp.write('| {}\n'.format(' '.join(text)))
    
# Создаем batches 
batches_path = f'data/batches/999'

if not os.path.exists(batches_path):
    os.makedirs(batches_path)
    
bv = artm.BatchVectorizer(data_path=vwpath,data_format='vowpal_wabbit',target_folder=batches_path)

CPU times: user 8.12 ms, sys: 90 µs, total: 8.21 ms
Wall time: 13.5 ms


In [16]:
# Матрица для  нового текста
theta = model.transform(batch_vectorizer=bv)
theta.T

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14
0,0.0,0.015097,0.022988,0.0,0.0,0.0,0.239324,0.0,0.0,0.405392,0.0,0.187578,0.0,0.129622,0.0


In [17]:
# Матрица модели для предсказания похожих новостей
theta_join = model.get_theta().T.reset_index()
del theta_join['index']
theta_join.head()

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14
0,0.196016,0.315601,0.000000,0.000000,0.000000,0.433173,0.000000,0.000000,0.000000,0.055209,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.341485,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.623804,0.000000,0.034712,0.000000,0.000000
2,0.000000,0.000000,0.037303,0.000000,0.859281,0.000000,0.000000,0.000000,0.000000,0.042286,0.000000,0.061129,0.000000,0.000000,0.000000
3,0.103273,0.000000,0.105823,0.114913,0.000000,0.119416,0.017386,0.306959,0.000000,0.005016,0.012834,0.000000,0.060301,0.000000,0.154078
4,0.028877,0.000000,0.232749,0.000000,0.000000,0.000000,0.110474,0.051190,0.077827,0.164708,0.013326,0.122516,0.123363,0.041254,0.033717


In [18]:
# Соединяем  текст и theta
joined = df.join(theta_join)
joined.head(2)

,year,month,text,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14
docID,,,,,,,,,,,,,,,,,,
0,2008,1,C 1 января 2008 года на территории Азербайджан...,0.196016,0.315601,0.0,0.000000,0.0,0.433173,0.0,0.0,0.0,0.055209,0.000000,0.0,0.000000,0.0,0.0
1,2008,1,Специалисты по связям с общественностью америк...,0.000000,0.000000,0.0,0.341485,0.0,0.000000,0.0,0.0,0.0,0.000000,0.623804,0.0,0.034712,0.0,0.0


In [20]:
# Самый ближайший топик для нашей новой новости
max_col =  theta.idxmax()[0]
phi = model.get_phi()
phi['word'] = phi.index
phi['word'] = phi.word.apply(lambda x: x[1])

print(f"{max_col}: {phi[[max_col, 'word']].sort_values(by=max_col, ascending=False)['word'].values[:15]}")

topic_9: ['фотография' 'девушка' 'внимание' 'женщина' 'снимка' 'блогер' 'мужчина'
 'комментарий' 'ребёнок' 'фото' 'написать' 'пост' 'обратить' 'очень'
 'автор']


In [21]:
# Пять ближайших  топиков для новой новости
cols = theta[0].sort_values(ascending=False).index[:5]
phi = model.get_phi()
phi['word'] = phi.index
phi['word'] = phi.word.apply(lambda x: x[1])

for col in cols:
    if col != 'word':
        print(f"{col}: {phi[[col, 'word']].sort_values(by=col, ascending=False)['word'].values[:10]}")

topic_9: ['фотография' 'девушка' 'внимание' 'женщина' 'снимка' 'блогер' 'мужчина'
 'комментарий' 'ребёнок' 'фото']
topic_6: ['ролик' 'фильм' 'реклама' 'youtube' 'видео' 'рекламный' 'игра' 'сериал'
 'просмотр' 'шоу']
topic_11: ['аккаунт' 'вконтакте' 'запись' 'блог' 'микроблог' 'популярный' 'друг'
 'фотография' 'instagram' 'администрация']
topic_13: ['new' 'times' 'книга' 'кандидат' 'выбор' 'обама' 'партия' 'номер'
 'основатель' 'post']
topic_2: ['британский' 'news' 'великобритания' 'бибись' 'bbc' 'daily' 'житель'
 'ребёнок' 'лондон' 'корпорация']


In [22]:
# Сортируем документы по  ближайшему топику к нашей новости
joined.sort_values(by=max_col,ascending=False).head(2)

,year,month,text,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14
docID,,,,,,,,,,,,,,,,,,
35831,2014,10,- Люблинский суд Москвы признал неправомерным...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.98408,0.0,0.015920,0.0,0.0,0.0
35549,2014,8,"– Украинский оператор мобильной связи \""МТС-У...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.012709,0.0,0.97389,0.0,0.013401,0.0,0.0,0.0


In [23]:
a = joined.sort_values(by=max_col,ascending=False)['text'][:10].values

In [41]:
# Выводим на печать 10 ближайших документов
for i in range (a.shape[0]):
    print (a[i][:600]+'\n')

 - Люблинский суд Москвы признал неправомерным возврат Роскомнадзором оппозиционеру Алексею Навальному документов на регистрацию информационного агентства и обязал федеральную службу повторно рассмотреть его заявление.\nКак передает корреспондент \"Интерфакса\" об этом говорится в постановлении судьи Юрия Лебедева оглашенном во вторник. Суд частично удовлетворил заявление Навального признав возврат документов на регистрацию СМИ неправомерным но отказав оппозиционеру в требовании обязать Роскомнадзор зарегистрировать его информационное агентство.\nСогласно постановлению Роскомнадзор должен в ус

 – Украинский оператор мобильной связи \"МТС-Украина\" сообщил что отключение абонентов в Симферополе связано с тем что компания лишилась технической возможности осуществлять контроль над одним из ключевых объектов связи который находится на территории Крыма.\n\"Компания предпринимает все возможные меры по выяснению причин происшествия и скорейшего восстановления связи. Приносим извинения абонен

### Косинусное расстояние

In [25]:
#Вектор новой новости
theta.T

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14
0,0.0,0.015097,0.022988,0.0,0.0,0.0,0.239324,0.0,0.0,0.405392,0.0,0.187578,0.0,0.129622,0.0


In [26]:
# Матрица новостей рубрики "Интернет и сми"
theta_join.head(5)

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14
0,0.196016,0.315601,0.000000,0.000000,0.000000,0.433173,0.000000,0.000000,0.000000,0.055209,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.341485,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.623804,0.000000,0.034712,0.000000,0.000000
2,0.000000,0.000000,0.037303,0.000000,0.859281,0.000000,0.000000,0.000000,0.000000,0.042286,0.000000,0.061129,0.000000,0.000000,0.000000
3,0.103273,0.000000,0.105823,0.114913,0.000000,0.119416,0.017386,0.306959,0.000000,0.005016,0.012834,0.000000,0.060301,0.000000,0.154078
4,0.028877,0.000000,0.232749,0.000000,0.000000,0.000000,0.110474,0.051190,0.077827,0.164708,0.013326,0.122516,0.123363,0.041254,0.033717


In [27]:
import scipy.spatial.distance
import numpy as np
from numpy.linalg import norm

In [28]:
u = theta_join.values
u1 = theta_join[:1].values
u1 = np.ravel(u1)

In [29]:
theta_cos = theta.T
v = theta_cos.values
v.shape
v = np.ravel(v)

In [30]:
#Проверяем в ручную на первом векторе из матрицы
cos_dot = np.dot(v, u1) / norm(u1) / norm(v)
1-cos_dot

0.9096071571111679

In [32]:
%%time
#  В атоматическом режиме
l = []
for i in range (u.shape[0]):
    cos = scipy.spatial.distance.cosine(v,u[i])
    l.append((cos,i))
    

CPU times: user 3.97 s, sys: 413 ms, total: 4.39 s
Wall time: 3.82 s


In [33]:
# Первые 10
l[:10]

[(0.9096071571111679, 0),
 (1.0, 1),
 (0.9348361045122147, 2),
 (0.9602210037410259, 3),
 (0.3437389135360718, 4),
 (0.7669801265001297, 5),
 (0.7533206641674042, 6),
 (0.9069559797644615, 7),
 (0.5452775657176971, 8),
 (0.5755695700645447, 9)]

In [34]:
%%time
# Отсортированы по возрастанию
sorted(l)[:10]

CPU times: user 69.6 ms, sys: 3.85 ms, total: 73.5 ms
Wall time: 72 ms


[(0.0037270784378051758, 17459),
 (0.015184938907623291, 37281),
 (0.01630300283432007, 4009),
 (0.01860886812210083, 17258),
 (0.02217942476272583, 11061),
 (0.023261427879333496, 38041),
 (0.02491682767868042, 32556),
 (0.026662707328796387, 37373),
 (0.028628170490264893, 36271),
 (0.028631269931793213, 38000)]

### Несколько примеров самых близких новостей

In [35]:
joined.iloc[17459,2][:500]

'Исследователи из американской компании по информационной безопасности FireEye заявили, что российские хакеры взломали компьютеры Белого дома, используя сервис микроблогов Twitter. Об этом сообщает Reuters. В американской фирме считают, что хакеры создали вредоносный код Hammertoss, распространявшийся через популярную соцсеть. Специальный алгоритм каждый день генерировал новое сообщение на платформе Twitter с изображением и ссылкой, которая вела к серверу с вирусом. Также в твите присутствовал хэ'

In [36]:
joined.iloc[37281,2][:500]

' - Стартовавший в минувшую субботу на \\"Первом канале\\" документальный сериал \\"Следствие покажет\\" который ведет представитель СКР Владимир Маркин занял первое место среди передач в вечерний прайм-тайм.\\nСогласно данным международной исследовательской группы \\"ТНС Гэллап Медиа\\" доля аудитории сериала составила 129% среди всех передач во временном сегменте с 17:40 до 18:40 когда на телеканалах в основном идут новостные и развлекательные передачи.\\nКомментируя ситуацию Маркин заявил \\"Интерфаксу'

In [37]:
joined.iloc[4009,2][:500]

'Итальянские власти обвинили четырех топ-менеджеров Google в нарушении закона о неприкосновенности частной жизни, пишет The International Herald Tribune. По мнению сотрудников миланской прокуратуры, сотрудники поисковика виноваты в том, что допустили размещение на итальянском видеохостинге Google Video оскорбительного ролика. В число обвиняемых попали старший вице-президент Google Дэвид Драммонд (David Drummond), бывший финдиректор компании Джордж Де Лос Рейес (George De Los Reyes), главный юриск'

In [38]:
text = '''Японский геймдизайнер Хидэо Кодзима опубликовал фотографию водки,чем привлек 
внимание русскоязычных комментаторов. Пост с алкоголем глава студии по разработке компьютерных игр 
выложил в Instagram.Россияне оставили множество комплиментов на странице Кодзимы. Большинство комментариев
содержали слова «гений» и «гениальный», отсылая к мему про геймдизайнера.Юзеры также начали использовать эпитет
«царь» по отношению к основателю игровой студии из-за марки водки на снимке.Один из пользователей с ником rodion_obrien
предложил выдвинуть Кодзиму в президенты. Некоторые комментаторы предположили, что фотографии — это намек на появление
в будущих играх русской озвучки.Накануне, 25 февраля, Кодзима опубликовал в Twitter несколько снимков с подарками от
российского иллюстратора Ильи Кувшинова. Тот посетил студию Kojima Productions и одарил ее сотрудников картинами своего
авторства.В январе 2018 года Кодзима создал аккаунт в 
Instagram, после этого русскоязычные юзеры оставили под публикациями похвалу в адрес геймдизайнера.'''